In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor, Resize, Compose, v2
from torchvision.datasets import Cityscapes
from torch.utils.data import random_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import utils
import wandb



In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: svenbbs. Use `wandb login --relogin` to force relogin


True

In [3]:
lr_rate = 2e-3
epochs = 10
batch_size = 64
subsize = (128, 256)
momentum = 0.9

In [4]:
augmented_transform = transforms.Compose([
    v2.Resize(subsize),
    v2.RandomHorizontalFlip(1),  # Random horizontal flip
    v2.RandomRotation(10),  # Random rotation up to 10 degrees
    v2.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color jitter
    v2.ToTensor()
])

augmented_target_transform = transforms.Compose([
    v2.Resize(subsize),
    v2.RandomHorizontalFlip(1),  # Same transformation as input images
    v2.RandomRotation(10),  # Same transformation as input images
    v2.ToTensor()
])

normal_transform = transforms.Compose([
    v2.Resize(subsize),
    v2.ToTensor()
])
normal_target_transform = transforms.Compose([
    v2.Resize(subsize),
    v2.ToTensor()
])


#Desktop
normal_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=normal_transform, target_transform=normal_target_transform)
augmented_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=augmented_transform, target_transform=augmented_target_transform)
dataset = torch.utils.data.ConcatDataset([normal_dataset, augmented_dataset])


#Laptop
#dataset = Cityscapes(root="C:/Users/20182573/Documents/CityScapes", split='train', mode='fine', target_type='semantic', transform=transform, target_transform=target_transform)

#subset_small, subset_big = random_split(dataset, [0.2,0.8])
train_dataset, val_dataset = random_split(dataset, [0.8,0.2])

<>:27: SyntaxWarning: invalid escape sequence '\C'
<>:28: SyntaxWarning: invalid escape sequence '\C'
<>:27: SyntaxWarning: invalid escape sequence '\C'
<>:28: SyntaxWarning: invalid escape sequence '\C'
C:\Users\Sven\AppData\Local\Temp\ipykernel_18072\3308899275.py:27: SyntaxWarning: invalid escape sequence '\C'
  normal_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=normal_transform, target_transform=normal_target_transform)
C:\Users\Sven\AppData\Local\Temp\ipykernel_18072\3308899275.py:28: SyntaxWarning: invalid escape sequence '\C'
  augmented_dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=augmented_transform, target_transform=augmented_target_transform)
c:\Users\Sven\anaconda3\envs\torch\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.

In [5]:
len(train_dataset)

4760

In [6]:


train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [7]:

#from model import Model
#model = Model().cuda()

model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=False, progress=True, num_classes=34).cuda()
loss_fn = nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate, momentum=momentum)


c:\Users\Sven\anaconda3\envs\torch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Sven\anaconda3\envs\torch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [8]:
def train(dataloader, model, loss_fn, optimizer, run):
    """
    Train a model for 1 epoch.

    Params:
    - dataloader:   dataset to train on.
    - model:        the model object to be trained.
    - loss_fn:      the loss function.
    - optimizer:    the desired optimization.
    """
    size = len(dataloader.dataset)
    model.train() #Set the model to train mode
    for batch, (IMG,SEGM) in enumerate(dataloader):
        IMG = IMG.to('cuda')
        SEGM  = (SEGM*255).long().squeeze()     #*255 because the id are normalized between 0-1
        SEGM = utils.map_id_to_train_id(SEGM).to('cuda')
        
        #predict
        pred = model(IMG)['out']
        #Loss
        loss = loss_fn(pred, SEGM)
        

        #Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        #print loss during training
        loss, current = loss.item(), (batch + 1) * len(IMG)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        run.log({"train_loss": loss})
    
    return run
            


In [9]:
def test(dataloader, model, loss_fn):
    """
    Test a model.

    Params:
    - dataloader:   dataset to test on.
    - model:        the model object to be tested.
    - loss_fn:      the loss function.
    """
    num_batches = len(dataloader)
    model.eval() #model in eval mode
    test_loss = 0
    with torch.no_grad():
        for _, (IMG,SEGM) in enumerate(dataloader):
            IMG = IMG.to('cuda')
            SEGM  = (SEGM*255).long().squeeze()     #*255 because the id are normalized between 0-1
            SEGM = utils.map_id_to_train_id(SEGM).to('cuda')

            pred = model(IMG)['out']
            test_loss += loss_fn(pred, SEGM).item()
            
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")
    return test_loss
    


In [10]:



run = wandb.init(
    # Set the project where this run will be logged
    project="CS_challenge", name="2.0",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": {lr_rate},
        "epochs": {epochs},
        "Momentum": {momentum},
        "Batch_size": {batch_size},
        "model version": 1.0,
        "subset size [%]": 100,
        "resize": {subsize},
    },
)




for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    run = train(train_dataloader, model, loss_fn, optimizer, run)
    test_loss = test(val_dataloader, model, loss_fn)
    run.log({"Test_loss": test_loss, "Epoch": t})
print("Done!")


Epoch 1
-------------------------------
loss: 3.644441  [   64/ 4760]
loss: 3.621849  [  128/ 4760]
loss: 3.567999  [  192/ 4760]
loss: 3.520430  [  256/ 4760]
loss: 3.426957  [  320/ 4760]
loss: 3.336647  [  384/ 4760]
loss: 3.231122  [  448/ 4760]
loss: 3.137012  [  512/ 4760]
loss: 3.007363  [  576/ 4760]
loss: 2.866226  [  640/ 4760]
loss: 2.756863  [  704/ 4760]
loss: 2.596910  [  768/ 4760]
loss: 2.479400  [  832/ 4760]
loss: 2.401435  [  896/ 4760]
loss: 2.266798  [  960/ 4760]
loss: 2.142769  [ 1024/ 4760]
loss: 2.027126  [ 1088/ 4760]
loss: 1.952096  [ 1152/ 4760]
loss: 1.864465  [ 1216/ 4760]
loss: 1.772544  [ 1280/ 4760]
loss: 1.702581  [ 1344/ 4760]
loss: 1.717569  [ 1408/ 4760]
loss: 1.674434  [ 1472/ 4760]
loss: 1.503769  [ 1536/ 4760]
loss: 1.527136  [ 1600/ 4760]
loss: 1.537006  [ 1664/ 4760]
loss: 1.471014  [ 1728/ 4760]
loss: 1.409607  [ 1792/ 4760]
loss: 1.443242  [ 1856/ 4760]
loss: 1.468837  [ 1920/ 4760]
loss: 1.474934  [ 1984/ 4760]
loss: 1.490449  [ 2048/ 4760]


KeyboardInterrupt: 

In [11]:
run.finish()

Epoch,▁▃▆█
Test_loss,█▄▂▁
train_loss,█▇▅▃▃▂▂▂▂▂▂▂▁▂▂▂▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Epoch,3
Test_loss,0.81594
train_loss,0.84674


In [12]:
torch.save(model.state_dict(), "model.pth")